In [1]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import scipy.stats as stat
import random
import diff_classifier.aws as aws
import diff_classifier.msd as msd
from diff_classifier.msd import binning, precision_weight, precision_averaging, plot_all_experiments
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm

In [3]:
to_track

['100nm_10k_PEG_XY01',
 '100nm_10k_PEG_XY02',
 '100nm_10k_PEG_XY03',
 '100nm_10k_PEG_XY04',
 '100nm_10k_PEG_XY05',
 '100nm_10k_PEG_XY06',
 '100nm_10k_PEG_XY07',
 '100nm_10k_PEG_XY08',
 '100nm_10k_PEG_XY09',
 '100nm_10k_PEG_XY10',
 '100nm_10k_PEG_XY11',
 '100nm_10k_PEG_XY12',
 '100nm_10k_PEG_XY13',
 '100nm_10k_PEG_XY14',
 '100nm_10k_PEG_XY15',
 '100nm_10k_PEG_XY16',
 '100nm_10k_PEG_XY17',
 '100nm_10k_PEG_XY18',
 '100nm_10k_PEG_XY19',
 '100nm_10k_PEG_XY20']

In [ ]:
folder = 'PS_1k_5k_10k_PEG_in_agarose'
bucket = 'dtoghani.data'
experiment = '100nm_10k_PEG' #Used for naming purposes. Should exclude XY and well information

vids = 20
to_track = []
wells = 4
frames = 651
fps = 100.02

for num in range(1, vids+1):
    to_track.append('{}_XY{}'.format(experiment, '%02d' % num)) #As per usual, use for loops to include all experiments

slices, bins, well_names = binning(to_track)
geoM2xy = {}
gSEM = {}
for sample_name in to_track:
    #aws.download_s3('{}/geomean_{}.csv'.format(folder, sample_name), 'geomean_{}.csv'.format(sample_name), bucket_name=bucket)
    #aws.download_s3('{}/geoSEM_{}.csv'.format(folder, sample_name), 'geoSEM_{}.csv'.format(sample_name), bucket_name=bucket)
    #geoM2xy[sample_name] = np.genfromtxt('geomean_{}.csv'.format(sample_name))
    #gSEM[sample_name] = np.genfromtxt('geoSEM_{}.csv'.format(sample_name))
    
    aws.download_s3('{}/msd_{}.csv'.format(folder, sample_name), 'msd_{}.csv'.format(sample_name), bucket_name=bucket)
    geoM2xy[sample_name], gSEM[sample_name] = msd.gavg_MSDs(sample_name, remote_folder=folder, bucket=bucket)

In [ ]:
for key in geoM2xy:
    geoM2xy[key][[geoM2xy][key]==0.0] = np.nan
    gSEM[key][[gSEM][key]==0.0] = np.nan

In [ ]:
geo_slices = {}
gSEM_slices = {}

w_slices = {}
wo_slices = {}

for key in bins:
    wo_slices[key] = precision_weight(bins[key], gSEM)
    geo_slices[key], gSEM_slices[key] = precision_averaging(bins[key], geoM2xy, gSEM, wo_slices[key]) 

for key in geo_slices:
    geo_slices[key][[geo_slices][key]==0.0] = np.nan
    gSEM_slices[key][[gSEM_slices][key]==0.0] = np.nan
    
weights = precision_weight(well_names, gSEM_slices)
geo, gS = precision_averaging(well_names, geo_slices, gSEM_slices, weights, bucket=bucket, folder=folder, experiment=experiment)

In [ ]:
to_plot_f = [experiment, experiment, experiment, experiment, experiment, experiment]
plot_all_experiments(to_plot_f, bucket=bucket, folder=folder)